In [1]:
import os,sys
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd
import pygeos
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon, mapping

from tqdm import tqdm
from pathlib import Path
import shapely

import networkx as nx

# sys.path.append(os.path.join('..','power_flow_model'))
# from simplify import *

current_dir = os.getcwd()
osm_flex_path = os.path.abspath(os.path.join(current_dir, '../../osm-flex/src'))
sys.path.insert(0, osm_flex_path)

import osm_flex.download as dl
import osm_flex.extract as ex
import osm_flex.config
import osm_flex.simplify as sy

In [2]:
osm_flex.config.OSM_CONFIG_FILE

PosixPath('/scistor/ivm/mye500/projects/osm-flex/src/osm_flex/osmconf.ini')

### Step 1: Extract power infrastructure data from OSM
Package: osm-flex

`vietnam-latest.osm.pbf` was downloaded on 24-10-2024.

In [3]:
# iso3 = 'VNM'
# dl.get_country_geofabrik(iso3)

In [5]:
path_vnm_dump = osm_flex.config.OSM_DATA_DIR.joinpath('vietnam-latest.osm.pbf')
gdf_vnm_power = ex.extract_cis(path_vnm_dump, 'power')

extract points: 100%|██████████| 86978/86978 [00:17<00:00, 5040.33it/s]
Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
extract multipolygons:   0%|          | 1/501 [00:19<2:44:21, 19.72s/it]Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
extract lines: 100%|██████████| 9166/9166 [00:15<00:00, 583.22it/s] 


#### Simplify OSM data

In [6]:
print(f'Number of results: {len(gdf_vnm_power)}')

gdf_vnm_power = sy.remove_contained_points(gdf_vnm_power)
print(f'Number of results after removing points contained in polygons: {len(gdf_vnm_power)}')

gdf_vnm_power = sy.remove_contained_polys(gdf_vnm_power)
print(f'Number of results after removing polygons contained in larger polygons: {len(gdf_vnm_power)}')

gdf_vnm_power = sy.remove_exact_duplicates(gdf_vnm_power)
print(f'Number of results after removing exact geometrical duplicates: {len(gdf_vnm_power)}')

Number of results: 96645
Number of results after removing points contained in polygons: 95572
Number of results after removing polygons contained in larger polygons: 95565


/scistor/ivm/mye500/miniconda3/envs/py310/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


Number of results after removing exact geometrical duplicates: 95565


In [7]:
gdf_vnm_power.to_file("../data/osm/vietnam-latest_power.gpkg", driver="GPKG")

In [8]:
osm_plants = gdf_vnm_power[gdf_vnm_power['power'] == 'plant']
print(len(osm_plants))

453


In [9]:
subs = gdf_vnm_power[gdf_vnm_power['power'] == 'substation']
print(len(subs))
subs['osm_id'].isnull().any()

1108


True

In [10]:
"""
    Problem: There are some substations extracted from osm.pbf file by osm_flex don't have osm_id.
    Solution:
        1. osm_subs: Load the GeoPackage containing substations extracted using QGIS QuickOSM tool - 'power_substation_vietnam.gpkg',
            including substations that are of type 'MultiPolygon'.
        2. Use spatial join to find matching geometries between subs and osm_subs.
        3. Add the matched osm_id from osm_subs to the subs DataFrame
"""
osm_subs = gpd.read_file('../data/osm/power_substation_vietnam.gpkg')

if subs.crs != osm_subs.crs:
    osm_subs = osm_subs.to_crs(subs.crs)

# Use spatial join to find matching geometries between `subs` and `osm_subs`.
matched = gpd.sjoin(subs, osm_subs[['geometry', 'osm_id']], op='intersects')

# Add the matched `osm_id` from `osm_subs` to the `osm_id` column of `subs`.
subs.loc[matched.index, 'osm_id'] = matched['osm_id_right']

/scistor/ivm/mye500/miniconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [11]:
subs['osm_id'].isnull().any()

False

In [11]:
subs.head()

,osm_id,power,voltage,utility,name,plant_output_electricity,plant_source,plant_method,generator_output_electricity,generator_source,generator_type,substation,location,cables,circuits,line,layer,geometry
12524,4926248381,substation,None,None,DCL 131-14,None,None,None,None,None,None,None,None,None,None,None,None,POINT (108.27591 11.78394)
13001,5003536554,substation,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (105.72276 10.05084)
13003,5003536582,substation,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (106.56409 11.02383)
27862,6924044149,substation,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (105.91132 21.06240)
49394,10837884264,substation,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (106.70632 10.77571)


In [13]:
subs.to_file("../data/osm/vietnam-latest_substation_supplyPolygonID.gpkg", driver="GPKG")

In [14]:
lines = gdf_vnm_power[gdf_vnm_power['power'] == 'line']
print(len(lines))
lines.head()

7776


,osm_id,power,voltage,utility,name,frequency,plant_output_electricity,plant_source,plant_method,generator_output_electricity,generator_source,generator_type,substation,location,cables,circuits,line,layer,geometry
86400,153608701,line,110000,None,None,50,None,None,None,None,None,None,None,None,6,2,None,None,"LINESTRING (106.06478 20.93373, 106.06483 20.9..."
86404,176708144,line,110000,None,None,50,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (106.58306 11.56381, 106.58077 11.5..."
86410,241194661,line,220000,None,None,50,None,None,None,None,None,None,None,None,6,2,None,None,"LINESTRING (106.75171 20.93445, 106.75168 20.9..."
86412,241255651,line,110000,None,None,50,None,None,None,None,None,None,None,None,None,None,None,None,"LINESTRING (106.77678 20.82928, 106.77699 20.8..."
86413,241255652,line,220000,None,None,50,None,None,None,None,None,None,None,None,6,None,None,None,"LINESTRING (106.77506 20.82938, 106.77325 20.8..."


In [15]:
lines.to_file("../data/osm/vietnam-latest_lines.gpkg", driver="GPKG")